In [37]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

pd.options.display.max_columns, pd.options.display.max_rows = 500, 200
pd.options.display.float_format = '{:.3f}'.format
import joblib
import warnings
warnings.filterwarnings('ignore')

from soccer.outcomes_features import TrainCreator
from soccer.preprocessing import DataPreprocessor
from soccer.outcomes_catboost import OutcomesCatBoost
from soccer.outcomes_lgbm import OutcomesLGBM

In [67]:
matches = DataPreprocessor(is_train=False).preprocessing()

In [68]:
train, validation, test = TrainCreator().train_validation_test(matches)

In [69]:
OutcomesCatBoost().cross_val_score()

Training on fold [0/3]

bestTest = 0.881852265
bestIteration = 853

Training on fold [1/3]

bestTest = 0.8816533872
bestIteration = 987

Training on fold [2/3]

bestTest = 0.8808610051
bestIteration = 888



0.881469924486442

In [33]:
OutcomesCatBoost().save_model()

In [70]:
validation = OutcomesCatBoost().validation()

MSE:  1.4481320772368373
MAE:  0.9390377800920607


In [28]:
# experiments = OutcomesCatBoost().optuna_optimization(100)

In [71]:
predictions = OutcomesCatBoost().predict()
predictions.head()

,match_id,home_goals,away_goals,home_win,draw,away_win
0,2165,1.330,1.508,0.335,0.250,0.415
1,2166,2.052,0.948,0.629,0.205,0.166
2,2459,1.469,1.060,0.466,0.261,0.273
3,2460,1.370,0.994,0.455,0.273,0.273
4,2461,1.080,0.889,0.396,0.309,0.295


In [72]:
matches = DataPreprocessor(is_train=False).preprocessing()
train, validation, test = TrainCreator().train_validation_test(matches)

In [97]:
OutcomesLGBM().cross_val_score()

0.880711042192055

In [74]:
OutcomesLGBM().save_model()

[LightGBM] [Warning] bagging_freq is set=50, subsample_freq=0 will be ignored. Current value: bagging_freq=50
[LightGBM] [Warning] lambda_l1 is set=1.11749, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.11749
[LightGBM] [Warning] lambda_l2 is set=3.50433, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.50433
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=50, subsample_freq=0 will be ignored. Current value: bagging_freq=50
[LightGBM] [Warning] lambda_l1 is set=1.11749, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.11749
[LightGBM] [Warning] lambda_l2 is set=3.50433, reg_lambda=0.0 will be ignored. Curr

In [98]:
validation = OutcomesLGBM().validation()
validation.head()

[LightGBM] [Warning] lambda_l1 is set=1.11749, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.11749
[LightGBM] [Warning] bagging_freq is set=50, subsample_freq=0 will be ignored. Current value: bagging_freq=50
[LightGBM] [Warning] lambda_l2 is set=3.50433, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.50433
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
MSE:  1.4311947585596805
MAE:  0.9337249958930501


,is_home,is_pandemic,avg_scoring_5,avg_scoring_10,avg_scoring_20,avg_scoring_30,avg_scoring_5_against,avg_scoring_10_against,avg_scoring_20_against,avg_scoring_30_against,tournament_type,tournament,league,opp_league,location_mean_score_5,location_mean_score_10,location_mean_score_20,location_mean_score_30,location_median_score_5,location_median_score_10,location_median_score_20,location_median_score_30,location_mean_score_5_against,location_mean_score_10_against,location_mean_score_20_against,location_median_score_10_against,location_max_score_10,score,prediction
0,True,0,2.200,1.500,1.800,1.633,1.200,0.900,1.450,1.533,1,Uruguay. First,Uruguay. First,Uruguay. First,3.000,2.700,1.950,1.667,2.000,2.500,1.000,1.000,1.000,1.600,1.350,1.500,7.000,1,1.649
1,True,0,1.200,1.100,0.800,1.067,0.800,1.000,1.050,1.000,1,Uruguay. First,Uruguay. First,Uruguay. First,1.200,1.700,1.400,1.400,1.000,1.500,1.000,1.000,1.200,1.600,1.300,2.000,3.000,0,1.167
2,True,0,1.000,1.100,0.800,0.867,1.000,0.700,0.850,0.967,1,Uruguay. First,Uruguay. First,Uruguay. First,1.000,1.000,1.100,1.167,1.000,1.000,1.000,1.000,1.200,1.900,1.600,2.000,2.000,1,1.095
3,True,0,1.400,0.900,1.100,1.333,0.600,1.200,1.200,1.200,1,Uruguay. First,Uruguay. First,Uruguay. First,0.800,0.900,1.100,1.533,1.000,1.000,1.000,1.000,0.400,0.500,0.650,0.000,2.000,1,1.292
4,True,0,1.200,1.300,1.500,1.400,1.200,1.200,1.250,1.367,1,Uruguay. First,Uruguay. First,Uruguay. First,2.000,1.600,1.600,1.500,1.000,1.500,1.000,1.000,1.000,1.500,1.450,2.000,4.000,0,1.420


In [ ]:
experiments = OutcomesLGBM().optuna_optimization(200)

[I 2023-09-23 16:03:23,864] A new study created in memory with name: no-name-8fcfd4ff-ed1a-4cc2-8c71-a9f2db235947


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-09-23 16:04:08,339] Trial 0 finished with value: 0.8800511981808229 and parameters: {'learning_rate': 0.022712999999999997, 'feature_fraction': 0.65, 'num_leaves': 40, 'lambda_l1': 2.83243, 'lambda_l2': 2.26221, 'bagging_fraction': 0.9299999999999999, 'bagging_freq': 128}. Best is trial 0 with value: 0.8800511981808229.
[I 2023-09-23 16:04:44,251] Trial 1 finished with value: 0.8801123071219513 and parameters: {'learning_rate': 0.027017, 'feature_fraction': 0.794, 'num_leaves': 33, 'lambda_l1': 2.1945500000000004, 'lambda_l2': 3.4118000000000004, 'bagging_fraction': 0.822, 'bagging_freq': 112}. Best is trial 0 with value: 0.8800511981808229.
[I 2023-09-23 16:05:39,720] Trial 2 finished with value: 0.8797074333989379 and parameters: {'learning_rate': 0.014926, 'feature_fraction': 0.722, 'num_leaves': 26, 'lambda_l1': 1.44441, 'lambda_l2': 4.27483, 'bagging_fraction': 0.786, 'bagging_freq': 76}. Best is trial 2 with value: 0.8797074333989379.
[I 2023-09-23 16:06:16,303] Trial 3 f

In [83]:
predictions = OutcomesLGBM().predict()
predictions.head()

[LightGBM] [Warning] lambda_l1 is set=1.11749, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.11749
[LightGBM] [Warning] bagging_freq is set=50, subsample_freq=0 will be ignored. Current value: bagging_freq=50
[LightGBM] [Warning] lambda_l2 is set=3.50433, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.50433
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


,match_id,home_goals,away_goals,home_win,draw,away_win
0,2165,1.766,1.315,0.481,0.233,0.286
1,2166,1.686,1.145,0.500,0.241,0.259
2,2459,1.395,1.305,0.392,0.258,0.350
3,2460,1.297,1.134,0.403,0.273,0.324
4,2461,1.249,1.041,0.410,0.282,0.307


In [92]:
from soccer.compare_models import compare_models

compare_models()

[LightGBM] [Warning] lambda_l1 is set=1.11749, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.11749
[LightGBM] [Warning] bagging_freq is set=50, subsample_freq=0 will be ignored. Current value: bagging_freq=50
[LightGBM] [Warning] lambda_l2 is set=3.50433, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.50433
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


,model,loss
0,Catboost,0.596
1,LightGBM,0.592
2,Modified Glicko-2,0.585
3,Original Glicko-2,0.593
